In [3]:
import pandas as pd
from model_tools import Preprocess
import os
from tqdm.notebook import tqdm

In [18]:
### format CC caltarget dataset
folder = 'C:\\Users\\ytsma22c\\Documents\\GitHub\\sensitivities\\CC caltarget sensitivity\\data'
file = 'caltarget_mean_spectra'
spectra = pd.read_csv(f'{folder}\\{file}.csv')

# SC resample
spectra_resampled = Preprocess.resample_to_match(spectra)
# BLR
spectra_blr = Preprocess.AirPLS(spectra_resampled)
# norm5
spectra_norm = Preprocess.norm5_SC(spectra_blr)

spectra_norm.to_csv(f'{folder}\\{file}_SC axis_AirPLS_norm5.csv', index=False)

In [6]:
## format pLIBS datasets
folder = 'P:\\pLIBS_DATASETS'
ftypes = ['10K axis_AirPLS_norm3', 'SC axis_AirPLS_norm5', 'AirPLS']
dtypes = ['average', 'location2']
axis_10k = pd.read_csv('P:\\DATASETS\\10K\\datasets for sensitivity calcs\\10K axis_AirPLS_norm3\\10K_duplicate_spectra_EARTH_5_formatted.csv', usecols=[0])

for dtype in tqdm(dtypes, desc='dataset'):
    
    file = f'pLIBS_Z300_spectra_{dtype}.csv'
    main_spectra = pd.read_csv(f'{folder}\\{file}')
    meta = pd.read_csv(f'{folder}\\pLIBS_Z300_metadata_{dtype}.csv')
    
    for ftype in tqdm(ftypes, desc='folders'):
        spectra = main_spectra.copy()
        
        if ftype == 'SC axis_AirPLS_norm5':
            # SC axis
            spectra_resampled = Preprocess.resample_to_match(spectra)
            # BLR
            spectra_blr = Preprocess.AirPLS(spectra_resampled)
            # norm5
            spectra_norm = Preprocess.norm5_SC(spectra_blr)
            # export
            spectra_norm.to_csv(f'{folder}\\{ftype}\\{file[:-4]}_formatted.csv', index=False)
            
        elif ftype == '10K axis_AirPLS_norm3':
            # SC axis
            spectra_resampled = Preprocess.resample_to_match(spectra, axis_10k)
            # BLR
            spectra_blr = Preprocess.AirPLS(spectra_resampled)
            # norm5
            spectra_norm = Preprocess.norm3_SL_10K(spectra_blr)
            # export
            spectra_norm.to_csv(f'{folder}\\{ftype}\\{file[:-4]}_formatted.csv', index=False)
            
        elif ftype == 'AirPLS':
            # BLR
            spectra_blr = Preprocess.AirPLS(spectra)
            # export
            spectra_blr.to_csv(f'{folder}\\{file[:-4]}_AirPLS.csv', index=False)

dataset:   0%|          | 0/2 [00:00<?, ?it/s]

folders:   0%|          | 0/3 [00:00<?, ?it/s]

Resampling to SuperCam -2px shifted wavelength axis by default


folders:   0%|          | 0/3 [00:00<?, ?it/s]

Resampling to SuperCam -2px shifted wavelength axis by default


In [17]:
### do L1 norm manually just because
spectra = pd.read_csv(f'{folder}\\pLIBS_Z300_spectra_location2_AirPLS.csv')
norm_dict = dict()
for s in tqdm(spectra.columns[1:]):
    total = spectra[s].sum()
    normed_spectrum = list(spectra[s]/total)
    norm_dict[s] = normed_spectrum
    
normed_spectra = pd.DataFrame.from_dict(norm_dict)
normed_spectra.insert(0,'wave',spectra['wave'])
normed_spectra.to_csv(f'{folder}\\pLIBS_Z300_spectra_location2_AirPLS_L1.csv')

  0%|          | 0/2922 [00:00<?, ?it/s]

In [24]:
## format CL datasets
folder = 'P:\\ChemLIBS_DATASETS\\DEVAS format datasets'
ftypes = ['SC axis_AirPLS_norm5', 'AirPLS_norm3CL','10K axis_AirPLS_norm3']
file_list = [f for f in os.listdir(folder) if 'spectra' in f]
axis_10k = pd.read_csv('P:\\DATASETS\\10K\\datasets for sensitivity calcs\\10K axis_AirPLS_norm3\\10K_duplicate_spectra_EARTH_5_formatted.csv', usecols=[0])

for file in tqdm(file_list[1:]):
    main_spectra = pd.read_csv(f'{folder}\\{file}')
    len1 = len(main_spectra)
    main_spectra.drop(index=[2047,4095], inplace=True)
    len2 = len(main_spectra)
    
    if main_spectra.isnull().values.any():
        print(file, 'na still there')
        break
    elif len1 - len2 != 2:
        print(file, 'wrong number of rows remaining')
        break
    else:
        main_spectra.to_csv(f'{folder}\\{file}', index=False)
    
    for ftype in tqdm(ftypes, leave=False):
        spectra = main_spectra.copy()
        
        if ftype == 'SC axis_AirPLS_norm5':
            # SC axis
            spectra_resampled = Preprocess.resample_to_match(spectra)
            # BLR
            spectra_blr = Preprocess.AirPLS(spectra_resampled)
            # norm5
            spectra_norm = Preprocess.norm5_SC(spectra_blr)
            # export
            spectra_norm.to_csv(f'{folder}\\{ftype}\\{file[:-4]}_formatted.csv', index=False)
            
        elif ftype == 'AirPLS_norm3CL':
            # BLR
            spectra_blr = Preprocess.AirPLS(spectra)
            # norm5
            spectra_norm = Preprocess.norm3_CL(spectra_blr)
            # export
            spectra_norm.to_csv(f'{folder}\\{ftype}\\{file[:-4]}_formatted.csv', index=False)
            
        elif ftype == '10K axis_AirPLS_norm3':
            # SC axis
            spectra_resampled = Preprocess.resample_to_match(spectra, axis_10k)
            # BLR
            spectra_blr = Preprocess.AirPLS(spectra_resampled)
            # norm5
            spectra_norm = Preprocess.norm3_SL_10K(spectra_blr)
            # export
            spectra_norm.to_csv(f'{folder}\\{ftype}\\{file[:-4]}_formatted.csv', index=False)

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Resampling to SuperCam -2px shifted wavelength axis by default


  0%|          | 0/3 [00:00<?, ?it/s]

Resampling to SuperCam -2px shifted wavelength axis by default


...

Resampling to SuperCam -2px shifted wavelength axis by default


  0%|          | 0/3 [00:00<?, ?it/s]

Resampling to SuperCam -2px shifted wavelength axis by default


  0%|          | 0/3 [00:00<?, ?it/s]

Resampling to SuperCam -2px shifted wavelength axis by default


In [25]:
## make datasets for locations 4 and for average spectra
ftypes = ['SC axis_AirPLS_norm5', 'AirPLS_norm3CL','10K axis_AirPLS_norm3']

for sfile in tqdm(file_list):
    
    mfilename = sfile.replace('spectra','metadata')
    sfilename = f'{sfile[:-4]}_formatted.csv'
    
    meta = pd.read_csv(f'{folder}\\{mfilename}')
    meta_loc4 = meta[meta['Location #']==4].reset_index(drop=True)

    for ftype in tqdm(ftypes):
        
        # export meta bc its not there yet
        meta.to_csv(f'{folder}\\{ftype}\\{mfilename}', index=False)
        meta_loc4.to_csv(f'{folder}\\{ftype}\\{mfilename[:-4]}_location4.csv', index=False)
        
        spectra = pd.read_csv(f'{folder}\\{ftype}\\{sfilename}')
        cols = list(meta_loc4.pkey.values)
        cols.insert(0,'wave')
        spectra_loc4 = spectra[cols]
        spectra_loc4.to_csv(f'{folder}\\{ftype}\\{sfilename[:-4]}_location4.csv', index=False)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

C:\Users\ytsma22c\Anaconda3\envs\auto\lib\site-packages\IPython\core\interactiveshell.py:3378: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
folder = 'C:\\Users\\ytsma22c\\Documents\\GitHub\\sensitivities\\SC sensitivity Mars SCCTs\\data'
spectra = pd.read_csv(f'{folder}\\scct_spectra.csv')

# double check matches axis
spectra_resampled = Preprocess.resample_to_match(spectra)

# BLR
spectra_blr = Preprocess.AirPLS(spectra_resampled)

# norm5
spectra_norm = Preprocess.norm5_SC(spectra_blr)
spectra_norm.to_csv(f'{folder}\\scct_spectra_formatted.csv', index=False)

Resampling to SuperCam -2px shifted wavelength axis by default


In [3]:
# convert 18K to 10K axis
#axis_10K = pd.read_csv('P:\\DATASETS\\10K\\final_datasets\\10K_spectra_EARTH_9mJ.csv', usecols=[0])

folder = 'P:\\DATASETS\\10K\\datasets for sensitivity calcs\\just preprocessed'
outfolder = 'P:\\DATASETS\\10K\\datasets for sensitivity calcs\\10K axis_AirPLS_norm3'
file_list = [f for f in os.listdir(folder) if 'spectra' in f]
for file in tqdm(file_list):
    spectra = pd.read_csv(f'{folder}\\{file}')
    
    # BLR
    spectra_blr = Preprocess.AirPLS(spectra)

    # norm3
    spectra_norm = Preprocess.norm3_SL_10K(spectra_blr)
    spectra_norm.to_csv(f'{outfolder}\\{file[:-4]}_formatted.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
inst_list = ['10K']#,'18K']

for inst in inst_list:
    folder = f'P:\\DATASETS\\{inst}\\final_datasets'
    outfolder = f'{folder}\\10K axis_AirPLS_norm3'

    spec_files = [f for f in os.listdir(folder) if (('spectra' in f) and ('no_outliers' not in f))]

    for file in tqdm(spec_files):
        df = pd.read_csv(f'{folder}\\{file}')

        # not going to resample to min step first
        # BLR
        df_blr = Preprocess.AirPLS(df)

        # norm5
        df_norm = Preprocess.norm3_SL_10K(df_blr)

        df_norm.to_csv(f'{outfolder}\\{file[:-4]}_formatted.csv', index=False)

  0%|          | 0/16 [00:00<?, ?it/s]